# Toronto with Geocodes

1. [Import](#import)
2. [Data Loading and Preparation](#load)
3. [Map of Toronto and Neighborhoods](#map)
4. [Export](#export)

### <a id="import"> </a> Importing packages

In [1]:
import pandas as pd
# install packages if not installed
# !pip install geocoder 
import geocoder
# !pip install folium
import folium

### <a id="load"> </a> Data Loading and Preparation

The data saved to csv from previous step will be loaded into pandas dataframe

In [2]:
toronto_wiki_data = pd.read_csv('data/toronto_wiki_data.csv')
# Print first 5 rows od dataframe
toronto_wiki_data.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,6,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,8,M7A,Queen's Park,Queen's Park


Unnamed: 0 is the previous data index which is not needed anymore and thus will me removed

In [3]:
toronto_wiki_data.drop('Unnamed: 0', inplace=True, axis=1)
toronto_wiki_data.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


### <a id="geo"> </a> Adding latitide and longitude to the dataframe

Adding Latitude and Longitude columns to the new dataframe. They will be populated later on.

In [4]:
toronto_geo_data = toronto_wiki_data
toronto_geo_data['Latitude'] = None
toronto_geo_data['Longitude'] = None
toronto_geo_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,None,None
1,M4A,North York,Victoria Village,None,None
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",None,None
3,M6A,North York,"Lawrence Heights, Lawrence Manor",None,None
4,M7A,Queen's Park,Queen's Park,None,None


In [5]:
# Testing the geocoder for a M5G postcode
lat_lng_coords = None
while lat_lng_coords is None:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format("M5G"))
    lat_lng_coords = g.latlng

print(lat_lng_coords)

[43.65609081300005, -79.38492999999994]


We will now loop through the entries of the dataframe and add their latitude and longitude

In [6]:

for i in toronto_geo_data.index:
    lat_lng_coords = None
    postcode = toronto_geo_data.loc[i]['PostalCode']
    
    while lat_lng_coords is None:
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postcode))
        lat_lng_coords = g.latlng
        
    lat = lat_lng_coords[0]
    lon = lat_lng_coords[1]
    toronto_geo_data.loc[i]['Latitude'] = lat
    toronto_geo_data.loc[i]['Longitude'] = lon
    

In [7]:
toronto_geo_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7524,-79.3292
1,M4A,North York,Victoria Village,43.7306,-79.3133
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6503,-79.3592
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7233,-79.4513
4,M7A,Queen's Park,Queen's Park,43.6612,-79.3917


### <a id="map">  </a> Map of Toronto and Neighborhoods

In [9]:
g = geocoder.arcgis('Toronto, Ontario')
torloc = g.latlng
torloc

[43.648690000000045, -79.38543999999996]

In [10]:
torm = folium.Map(location=torloc, zoom_start=12)

In [11]:
radius = 5
for i in toronto_geo_data.index:
    lat = toronto_geo_data.loc[i]['Latitude']
    long = toronto_geo_data.loc[i]['Longitude']
    popup = toronto_geo_data.loc[i]['Neighborhood']
    folium.Marker(location=[lat,long], radius=radius, popup=popup).add_to(torm)
    

In [14]:
torm

### <a id="export"> </a> Exporting data to csv file for later usage

In [13]:
toronto_geo_data.to_csv('data/toronto_geo_data.csv')